# Load library

In [1]:
%pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.0 requires jinja2, which is not installed.
torch 2.2.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt 
import re, os, sys
import copy
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import pandas as pd
import random
import time
import psutil
from io import open
from sklearn.manifold import TSNE
from tensorflow import keras
from keras.models import Sequential
# from keras.layers.core import Dense
from keras.layers import Dense
from keras import backend as K
from os.path import exists
from pathlib import Path
from os.path import exists as file_exists
import sys
from sklearn.model_selection import train_test_split
import itertools
import psutil
from sklearn.metrics import silhouette_score, davies_bouldin_score, accuracy_score, homogeneity_score
from sklearn.metrics import completeness_score
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import confusion_matrix
from keras.layers import Dense
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
import csv
import math
from sklearn.decomposition import PCA
# from mpl_toolkits import mplot3d
from numpy.linalg import eig
from sklearn.datasets import make_blobs
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.datasets import make_swiss_roll
from sklearn.datasets import make_s_curve

2026-01-22 18:36:48.044879: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-22 18:36:48.069645: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-22 18:36:48.218191: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-22 18:36:48.218999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-22 18:36:48.889829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Utils Functions

In [3]:
def check_sequence(header, seq):
    """
    Adapted from VAMB: https://github.com/RasmussenLab

    Check that there're no invalid characters or bad format
    in the file.

    Note: The GAPS ('-') that are introduced from alignment
    are considered valid characters.
    """

    if len(header) > 0 and (header[0] in ('>', '#') or header[0].isspace()):
        raise ValueError('Bad character in sequence header')
    if '\t' in header:
        raise ValueError('tab included in header')

    basemask = bytearray.maketrans(b'acgtuUswkmyrbdhvnSWKMYRBDHV-',
                                   b'ACGTTTNNNNNNNNNNNNNNNNNNNNNN')

    masked = seq.translate(basemask, b' \t\n\r')
    stripped = masked.translate(None, b'ACGTN')
    if len(stripped) > 0:
        bad_character = chr(stripped[0])
        msg = "Invalid DNA byte in sequence {}: '{}'"
        raise ValueError(msg.format(header, bad_character))
    return masked

def get_class_ranges(df):
    class_ranges = {}
    labels = df['cluster_id'].unique()
    for label in labels:
        indices = df.index[df['cluster_id'] == label].tolist()
        start_idx = indices[0]
        end_idx = indices[-1]
        class_ranges[label] = (start_idx, end_idx)
    return class_ranges

def SummaryFasta(fname, GT_file=None):
    lines = list()
    seq_id = ""
    names, lengths = [], []
    ground_truth = None
    cluster_dis = None
    class_indices = {}

    if GT_file:
        ground_truth = []
        df = pd.read_csv(GT_file, sep='\t')
        print(df)
        GT_dict = dict(zip(df.sequence_id, df.cluster_id))
        cluster_dis = df['cluster_id'].value_counts().to_dict()

    for line in open(fname, "rb"):

        if line.startswith(b'#'):
            pass

        elif line.startswith(b'>'):
            if seq_id != "":
                seq = bytearray().join(lines)

                if (GT_file and not seq_id in GT_dict):
                    raise ValueError('Check GT for sequence {}'.format(seq_id))

                seq = check_sequence(seq_id, seq)
                names.append(seq_id)
                lengths.append(len(seq))

                if GT_file:
                    ground_truth.append(GT_dict[seq_id])
                    if GT_dict[seq_id] not in class_indices:
                        class_indices[GT_dict[seq_id]] = []
                    class_indices[GT_dict[seq_id]].append(len(names) - 1)

                lines = []
                seq_id = line[1:-1].decode()  # Modify this according to your labels.

            seq_id = line[1:-1].decode()

        else:
            lines += [line.strip()]

    if (GT_file and not seq_id in GT_dict):
        raise ValueError('Check GT for sequence {}'.format(seq_id))

    seq = bytearray().join(lines)
    seq = check_sequence(seq_id, seq)
    names.append(seq_id)
    lengths.append(len(seq))

    if GT_file:
        ground_truth.append(GT_dict[seq_id])
        if GT_dict[seq_id] not in class_indices:
            class_indices[GT_dict[seq_id]] = []
        class_indices[GT_dict[seq_id]].append(len(names) - 1)

    class_ranges =get_class_ranges(df)

    return names, lengths, ground_truth, cluster_dis, class_ranges

# 1. Feature Extraction: Kmer

In [4]:
def read_nucleotide_sequences(file):
    if os.path.exists(file) == False:
        print('Error: file %s does not exist.' % file)
        sys.exit(1)
    with open(file) as f:
        records = f.read()
    if re.search('>', records) == None:
        print('Error: the input file %s seems not in FASTA format!' % file)
        sys.exit(1)
    records = records.split('>')[1:]
    fasta_sequences = []
    for fasta in records:
        array = fasta.split('\n')
        header, sequence = array[0].split()[0], re.sub('[^ACGTU-]', '-', ''.join(array[1:]).upper())
        header_array = header.split('|')
        name = header_array[0]
        label = header_array[1] if len(header_array) >= 2 else '0'
        label_train = header_array[2] if len(header_array) >= 3 else 'training'
        sequence = re.sub('U', 'T', sequence)
        fasta_sequences.append([name, sequence, label, label_train])
    return fasta_sequences

def read_protein_sequences(file):
    if os.path.exists(file) == False:
        print('Error: file %s does not exist.' % file)
        sys.exit(1)
    with open(file) as f:
        records = f.read()
    if re.search('>', records) == None:
        print('Error: the input file %s seems not in FASTA format!' % file)
        sys.exit(1)
    records = records.split('>')[1:]
    fasta_sequences = []
    for fasta in records:
        array = fasta.split('\n')
        header, sequence = array[0].split()[0], re.sub('[^ACDEFGHIKLMNPQRSTVWY-]', '-', ''.join(array[1:]).upper())
        header_array = header.split('|')
        name = header_array[0]
        label = header_array[1] if len(header_array) >= 1 else '0'
        label_train = header_array[2] if len(header_array) >= 2 else 'training'
        fasta_sequences.append([name, sequence, label, label_train])
    return fasta_sequences

def readFasta(file):
    if os.path.exists(file) == False:
        print('Error: "' + file + '" does not exist.')
        sys.exit(1)

    with open(file) as f:
        records = f.read()

    if re.search('>', records) == None:
        print('The input file seems not in fasta format.')
        sys.exit(1)

    records = records.split('>')[1:]
    myFasta = []
    for fasta in records:
        array = fasta.split('\n')
        name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
        myFasta.append([name, sequence])
    return myFasta

# Write Kmer Files

In [5]:
import numpy as np
import pandas as pd


def write_to_svm(encodings, file):
    with open(file, 'w') as f:
        for line in encodings[1:]:
            line = line[1:]
            f.write('%s' % line[0])
            for i in range(1, len(line)):
                f.write('  %d:%s' % (i, line[i]))
            f.write('\n')


def write_to_tsv(encodings, file):
    with open(file, 'w') as f:
        for line in encodings[1:]:
            line = line[1:]
            f.write('%s' % line[0])
            for i in range(1, len(line)):
                f.write('\t%s' % line[i])
            f.write('\n')


def write_to_tsv_1(encodings, file):
    with open(file, 'w') as f:
        for line in encodings:
            f.write('%s' % line[0])
            for i in range(2, len(line)):
                f.write('\t%s' % line[i])
            f.write('\n')


def write_to_csv(encodings, file):
    with open(file, 'w') as f:
        for line in encodings[1:]:
            line = line[1:]
            f.write('%s' % line[0])
            for i in range(1, len(line)):
                f.write(',%s' % line[i])
            f.write('\n')


def write_to_weka(encodings, file):
    with open('%s.weka' % file, 'w') as f:
        f.write('@relation descriptor\n\n')
        for i in range(1, len(encodings[0][2:]) + 1):
            f.write('@attribute f.%d numeric\n' % i)
        f.write('@attribute play {yes, no}\n\n')
        f.write('@data\n')
        for line in encodings[1:]:
            line = line[1:]
            for fea in line[1:]:
                f.write('%s,' % fea)
            if line[0] == '1':
                f.write('yes\n')
            else:
                f.write('no\n')


def save_file(encodings, format='svm', file='encodings.txt'):
    if encodings == 0:
        with open(file, 'w') as f:
            f.write('An error encountered.')
    else:
        if format == 'svm':
            write_to_svm(encodings, file)
        if format == 'tsv':
            write_to_tsv(encodings, file)
        if format == 'csv':
            write_to_csv(encodings, file)
        if format == 'weka':
            write_to_weka(encodings, file)
        if format == 'tsv_1':
            write_to_tsv_1(encodings, file)


def save_cluster_result(cluster, e, file='Clusters.txt'):
    with open(file, 'w') as f:
        if cluster == 0:
            f.write(str(e))
        else:
            myCluster = np.array(cluster)
            df = pd.DataFrame({'name': myCluster[:, 0], 'cluster': myCluster[:, 1]})
            mySet = set(np.array(df.cluster).tolist())
            f.write('# The sample/feature can be clustered into %d clusters:\n' % len(mySet))
            for l in sorted(mySet):
                newData = np.array(df.loc[df.loc[:, "cluster"] == l, :].name)
                f.write('Cluster_%s:\t' % l)
                for i in newData:
                    f.write(i + '\t')
                f.write('\n')
            f.write('\n==============================================================\n')

            f.write('Protein/Feature\tcluster\n')
            for i in cluster:
                f.write(i[0] + '\t' + str(i[1]) + '\n')
    return None


def save_FS_result(feature, e, method, file='featureRank.txt'):
    with open(file, 'w') as f:
        if feature == 0:
            f.write(str(e))
        else:
            f.write('# Feature selection method: %s\n' % method)
            f.write(
                '# The features were ranked according to their importance, the topper the more important the feature is\n');
            f.write('=======================\n')
            for i in feature:
                f.write(i[0] + '\t' + str(i[1]) + '\n')
    return None


def save_reduction_result(reduced_data, file='dimension_reduction.txt'):
    with open(file, 'w') as f:
        f.write('sample')
        for i in range(1, len(reduced_data[0])):
            f.write('\tpc.' + str(i))
        f.write('\n')
        for i in reduced_data:
            f.write(i[0])
            for j in range(1, len(i)):
                f.write('\t' + str(i[j]))
            f.write('\n')
    return None


def save_CV_result_binary(data, out, info=None):
    with open(out, 'w') as f:
        if info:
            f.write('%s\n' % info)
        for i in range(len(data)):
            f.write('# result for fold %d\n' %(i + 1))
            for j in range(len(data[i])):
                f.write('%d\t%s\n' % (data[i][j][0], data[i][j][2]))
    return None

def save_CV_result(data, classes, out, info=None):
    with open(out, 'w') as f:
        if info:
            f.write('%s\n' % info)
        for i in range(len(data)):
            f.write('result for fold %d\n' %(i + 1))
            f.write('label')
            for k in classes:
                f.write('\t%s' %k)
            f.write('\n')
            for j in range(len(data[i])):
                f.write('%d' % data[i][j][0])
                for k in range(1, len(data[i][j])):
                    f.write('\t%s' %data[i][j][k])
                f.write('\n')
    return None


def save_IND_result_binary(data, out, info=None):
    with open(out, 'w') as f:
        if info:
            f.write('%s\n' % info)
        for i in data:
            f.write('%d\t%s\n' % (i[0], i[2]))
    return None

def save_IND_result(data, classes, out, info=None):
    with open(out, 'w') as f:
        if info:
            f.write('%s\n' % info)
        f.write('label')
        for k in classes:
            f.write('\t%s' % k)
        f.write('\n')
        for i in data:
            f.write('%d' %i[0])
            for j in range(1, len(i)):
                f.write('\t%s' %i[j])
            f.write('\n')
    return None


def save_prediction_metrics_ind(m_dict, out):
    with open(out, 'w') as f:
        f.write('#')
        for key in m_dict:
            f.write('\t%s' %key)
        f.write('\n')
        f.write('Indep')
        for key in m_dict:
            f.write('\t%s' %m_dict[key])
        f.write('\n')
    return None


def save_prediction_metrics_ind_muti(m_dict, classes, out):
    with open(out, 'w') as f:
        f.write('#')
        for c in classes:
            f.write('\tclass_%s_acc' %c)
        f.write('\n')
        f.write('Indep')
        for c in classes:
            f.write('\t%s' % m_dict[c])
        f.write('\n')
    return None


def save_prediction_metrics_cv(m_list, out):
    with open(out, 'w') as f:
        f.write('Fold')
        for key in m_list[0]:
            f.write('\t%s' %key)
        f.write('\n')
        for i in range(len(m_list)):
            f.write('%d' %(i + 1))
            for key in m_list[i]:
                f.write('\t%s' %m_list[i][key])
            f.write('\n')
    return None

def save_prediction_metrics_2_classes(m_dict, out):
    with open(out, 'w') as f:
        f.write('#')
        for m in ['Sensitivity', 'Specificity', 'Accuracy', 'MCC', 'Recall', 'Precision', 'F1-score']:
            f.write('\t%s' %m)
        f.write('\n')
        for key in m_dict:
            f.write('%s' %key)
            for m in ['Sensitivity', 'Specificity', 'Accuracy', 'MCC', 'Recall', 'Precision', 'F1-score']:
                f.write('\t%.3f' %m_dict[key][m])
            f.write('\n')
    return None

def save_prediction_metrics_cv_muti(m_list, classes, out):
    with open(out, 'w') as f:
        f.write('Fold')
        for c in classes:
            f.write('\tclass_%s_acc' %c)
        f.write('\n')
        for fold in range(len(m_list)):
            f.write('fold_%s' %(fold + 1))
            for c in classes:
                f.write('\t%s' %m_list[fold][c])
            f.write('\n')
    return None

def save_prediction_metrics_muti_V2(muti_record_metrics, out):
    with open(out, 'w') as f:
        f.write('#\tCV_accuracy\tIND_accuracy\n')
        for key in muti_record_metrics:
            f.write('%s\t%.4f\t%.4f\n' %(key, muti_record_metrics[key][0], muti_record_metrics[key][1]))

## Function for Generate K-mer

In [6]:
import pandas as pd
import numpy as np
import re
import itertools
from collections import Counter


def kmerArray(sequence, k):
    kmer = []
    for i in range(len(sequence) - k + 1):
        kmer.append(sequence[i:i + k])
    return kmer

def Kmer(fastas, k=2, type="DNA", upto=False, normalize=True, **kw):
    encoding = []
    lK = []
    header = ['#', 'label']
    NA = 'ACGT' if type in ("DNA", 'RNA') else 'ACDEFGHIKLMNPQRSTVWY'

    if k < 1:
        print('Error: the k-mer value should be larger than 0.')
        return 0

    if upto:
        for tmpK in range(1, k + 1):
            for kmer in itertools.product(NA, repeat=tmpK):
                header.append(''.join(kmer))
        encoding.append(header)
        for i in fastas:
            name, sequence, label = i[0], re.sub('-', '', i[1]), i[2]
            count = Counter()
            for tmpK in range(1, k + 1):
                kmers = kmerArray(sequence, tmpK)
                count.update(kmers)
                if normalize:
                    for key in count:
                        if len(key) == tmpK:
                            count[key] = count[key] / len(kmers)
            lK.append(kmers)
            code = [name, label]
            for j in range(2, len(header)):
                code.append(count[header[j]] if header[j] in count else 0)
            encoding.append(code)
    else:
        for kmer in itertools.product(NA, repeat=k):
            header.append(''.join(kmer))
        encoding.append(header)

        for i in fastas:
            name, sequence, label = i[0], re.sub('-', '', i[1]), i[2]
            kmers = kmerArray(sequence, k)
            count = Counter()
            count.update(kmers)
            if normalize:
                for key in count:
                    count[key] = count[key] / len(kmers)

            code = [name, label]
            for j in range(2, len(header)):
                code.append(count[header[j]] if header[j] in count else 0)
            encoding.append(code)

    dict = {'feature': header[2:]}
    df = pd.DataFrame(dict)
    df.to_csv('Feature.csv')
    return encoding

def preprocess(f):
    Feature_app1 = []
    for i in f:
        L = i.strip().split(':')
        Feature_app = [str(L[j]).split()[0] for j in range(1, len(L))]
        Feature_app1.append(Feature_app)
    Feature_app2 = np.array(Feature_app1).astype(float)
    return Feature_app2

# 2. Reduce Feature Dimension: PCA

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def run_pca(X,n_components):
    # Step 1: StandardScaler
    sc = StandardScaler()
    X_scale = sc.fit_transform(X)
    
    # Step 2: PCA
    pca = PCA(n_components=n_components)
    X_scale_pca = pca.fit_transform(X_scale)
  
    # Step 3: Get components
    n_pcs = pca.components_.shape[0]
    print("Number of principal components:", n_pcs)

    # Step 4: Select components with explained variance ratio > 0.1
    exp_var_pca = pca.explained_variance_ratio_
    total_explained_variance = exp_var_pca.sum()  

    return X_scale, X_scale_pca, pca.components_, exp_var_pca, total_explained_variance, pca

# 3. Create pattern for learning 

## Step 3.1: Generate synthetic data using make_blobs with a random number of clusters and standard deviation


###  Util function: writing training files

In [8]:
def Sort(sub_list):
 
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of
    # sublist lambda has been used
    return(sorted(sub_list, key = lambda x: x[0]))

def write_files(paths, features, number_dimension, j, i,n):
    dimension_list = []
    #normalized to 0-1
    for k in range(number_dimension):
        X = features[:, k]
        min_xvalue = min(X)
        if min_xvalue < 0:
            X = X + abs(min_xvalue)
        max_xvalue = max(X)
        X = X / abs(max_xvalue)
        dimension_list.append(X)

    number_point = len(dimension_list[0])
    file_name = f"train-cluster{j}-dataset{i}.txt"
    with open(f"{paths}/{file_name}", "w") as fout:
        for k in range(number_point):
            for l in range(number_dimension):
                print(dimension_list[l][k], end=" ", file=fout)
            output_values = [0] * n
            if 1 <= j <= n:
                output_values[j - 1] = 0.5
            print(" ".join(map(str, output_values)), file=fout)

### Gaussian Only

In [9]:
def Generate_guassian_make_bobs(start_gen_gaussian_train_data_set,end_gen_gaussian_train_data_set,num_real, start_cluster,max_number_of_cluster,min_no_point,max_no_point, dimension,min_std,max_std,n_real,paths):
    
    #Variable assignment    
    start_number_data_set   = start_gen_gaussian_train_data_set
    last_number_of_data_set = end_gen_gaussian_train_data_set
    min_no_point  = min_no_point
    max_no_point  = max_no_point
    number_dimension = dimension


    print("generate gaussian training data_real")
    print("from data set", start_number_data_set, "-", last_number_of_data_set)
    print("number dimension:",number_dimension)

    #----------------------------------------------------------------#
    for j in range(start_cluster,max_number_of_cluster+1):
        if j!=n_real: 
            for i in range(start_number_data_set,last_number_of_data_set+1):
                number_of_center = j
                number_point = random.randint(min_no_point, max_no_point)
                stdx= random.uniform(min_std,max_std)
                features, clusters, center = make_blobs(n_samples = number_point, \
                                                n_features = number_dimension, \
                                                centers = number_of_center,\
                                                cluster_std = stdx , shuffle = True, \
                                                return_centers = True)
                write_files(paths,features,number_dimension,j,i,max_number_of_cluster)
        else:
            for i in range(start_number_data_set,last_number_of_data_set-num_real+1):
                number_of_center = j
                number_point = random.randint(min_no_point, max_no_point)
                stdx= random.uniform(min_std,max_std)
                features, clusters, center = make_blobs(n_samples = number_point, \
                                                n_features = number_dimension, \
                                                centers = number_of_center,\
                                                cluster_std = stdx , shuffle = True, \
                                                return_centers = True)
                write_files(paths,features,number_dimension,j,i,max_number_of_cluster)
                

## Step 3.2: Generate patterns from dense regions using a random seed and nearest neighbors

In [10]:
import numpy as np
from scipy.stats import gaussian_kde
import random
import matplotlib.pyplot as plt

def select_random_high_density_point(data, labels, class_range):
    """
    Select a random point from the high-density region of each cluster.

    :param data: Numpy array of shape (n_samples, n_features)
    :param labels: Numpy array of shape (n_samples,), cluster labels for each sample
    :param class_range: Dictionary mapping class names to tuples of index ranges
    :return: List of random points from high-density regions, one for each cluster
    """
    selected_points = []
    centers = []
    high_density_points = []
    std_dev_c=[]
    
    for class_name, (start_idx, end_idx) in class_range.items():
        # Extract points belonging to the current class based on the index range
        cluster_points = data[start_idx:end_idx + 1]

        # If the cluster has fewer than 2 points, just select the point(s) directly
        if len(cluster_points) < 2:
            selected_points.extend(cluster_points)
            continue

        # Estimate the density of points in the cluster
        kde = gaussian_kde(cluster_points.T)
        
        # Evaluate the density at each point
        densities = kde(cluster_points.T)
        
        # Get the indices of the high-density points
        high_density_indices = np.where(densities > np.percentile(densities,85))[0]
        
        # Randomly select one point from the high-density region
        selected_index = random.choice(high_density_indices)
        selected_points.append(cluster_points[selected_index])

        # Compute the center of high-density region
        high_density_cluster_points = cluster_points[high_density_indices]
        ######## this is true std of cluster or not 
        # Calculate and print standard deviation of high-density points
        std_dev = np.std(high_density_cluster_points, axis=0)
        # print(f'Standard deviation of high-density points for class {class_name}: {std_dev}')
        std_dev_c.append(np.mean(std_dev))
        # print(std_dev_c)
        #####################
        high_density_points.extend(high_density_cluster_points)
        center = high_density_cluster_points.mean(axis=0)
        centers.append(center)

    # Convert high_density_points to numpy array
    high_density_points = np.array(high_density_points)

    # Convert selected_points and centers to numpy arrays
    selected_points = np.array(selected_points)
    centers = np.array(centers)
    
    return selected_points,std_dev_c


In [11]:
import numpy as np

def random_pick_and_find_nearby(X_normalized, class_ranges, selected_points,increase, mean_std_devs):
    # For each cluster, select a random seed point and collect neighboring points within a
    # radius defined by the cluster-wise mean standard deviation scaled by `increase`

    random_points = {}
    nearby_points_c = np.empty((0, X_normalized.shape[1]))
    i = 0
    
    for class_label, (start_idx, end_idx) in class_ranges.items():
        # 1. define radius
        radius = increase*mean_std_devs[i] 
         
        # 2. random point from selected points
        random_point = selected_points[i]
        random_points[class_label] = random_point
   
        # 3. find nearest distance
        distances = np.linalg.norm(X_normalized - random_point, axis=1)
        nearby_indices = np.where(distances < radius)[0]
        nearby_points = X_normalized[nearby_indices]

        # 4. Include the random point in the nearby points
        if not np.any(np.all(nearby_points == random_point, axis=1)):
            nearby_points = np.vstack((nearby_points, random_point))
        
        # 5. Concatenate nearest points
        nearby_points_c = np.concatenate((nearby_points_c, nearby_points), axis=0)
        i += 1
        
    return nearby_points_c

In [12]:
def Generate_real(start_gen_real_train_data_set, end_gen_real_train_data_set,X_normalized, class_ranges,n_real,GT,increase,max_number_of_cluster,paths):
  
    start_number_data_set = start_gen_real_train_data_set
    last_number_of_data_set = end_gen_real_train_data_set

    print("Generate Real training data")
    print(f"From dataset {start_number_data_set} - {last_number_of_data_set}")
    n_count=[]
    std_dev_collect=[]

    for i in range(start_number_data_set, last_number_of_data_set + 1):
      selected_points,std_dev_c= select_random_high_density_point(X_normalized, GT, class_ranges)
      nearby_points =random_pick_and_find_nearby(X_normalized, class_ranges, selected_points,increase,std_dev_c)
      n_count.append(len(nearby_points))
      std_dev_collect.append(std_dev_c)
      write_files(paths, nearby_points, X_normalized.shape[1], n_real, i,max_number_of_cluster)

    return n_count, std_dev_collect

# 4: Combine Gaussian and KDE-based training samples


In [13]:
import numpy as np

import sys
import csv
import copy
import random
import ast

def train_only_add_points_before_encode_data(start_add_point_train_data_set,end_add_point_train_data_set,start_cluster,max_number_of_cluster,max_no_point,number_dimension,paths):
    total_cluster = max_number_of_cluster
    add_x = 0.0
    add_y = 0.0


    bitlength = max_number_of_cluster
    train_start_dataset = start_add_point_train_data_set
    train_last_number_of_data_set=end_add_point_train_data_set

    print("add train - test point")
    print("number of encode train data in each cluster:", \
        train_last_number_of_data_set)

    #------------- find number of added lines in train data set -------------------
    max_line = 0
    min_line = 100000

    for j in range(start_cluster,max_number_of_cluster+1):
        for i in range(train_start_dataset, train_last_number_of_data_set+1):
            input_file_name = f"{paths}/train-cluster" + str(j) + "-dataset" + str(i) + ".txt"
            with open(input_file_name,'r') as data_file:
                number_line = 0
                for line in data_file:
                    data = line.split()
                    size = len(data)
                    if size > 0:
                        number_line = number_line + 1
                    else:
                        print("empty line: remove", number_line, " line from file") 
    #            number_line = len(data_file.readlines())
            data_file.close()
            
            if max_line < number_line:
                max_line = number_line
            if min_line > number_line:
                min_line = number_line
            
    train_max_point = max_line
    train_min_point = min_line
    max_point = train_max_point
    if max_point < max_no_point:
        max_point = max_no_point 

    print("train max point: ", max_point, " train min point: ", train_min_point)

    #------------------------ train data --------------------------------------------    
    print("add train point")
    train_data_set = []

    for j in range(start_cluster,max_number_of_cluster+1):
        for i in range(train_start_dataset, train_last_number_of_data_set+1):
            input_file_name = f"{paths}/train-cluster" + str(j) + "-dataset" + str(i) + ".txt"
            count_line = 0
            with open(input_file_name,'r') as data_file:
                for line in data_file:
                    data = line.split()
                    size = len(data)
                    if size > 0:
                        count_line = count_line + 1
                    else:
                        print("empty line: remove", count_line, " line from file")      
            data_file.close()

            if count_line < max_point:
                add_pattern = max_point - count_line
                train_data_set = []
                with open(input_file_name,'r') as data_file:
                    for line in data_file:
                        data = line.split()
                        size = len(data)
                        if size > 0:
                            cluster_name_bitpattern = []
                            for k in range(number_dimension):
                                x = float(data[k])
                                cluster_name_bitpattern.append(x)
                            target = []
                            for k in range(bitlength):
                                bitnumber = ast.literal_eval(data[k+number_dimension])
                                cluster_name_bitpattern.append(bitnumber)
                                target.append(bitnumber)
                            temp_list = copy.deepcopy(cluster_name_bitpattern)
                            train_data_set.append(temp_list)
                        else:
                            print("last line is empty: delete empty line")
                data_file.close()

                with open(input_file_name,'a') as data_file:
                    for k in range(add_pattern):
                        index = random.randint(0,count_line-1)
                        for l in range(number_dimension):
                            add_x = train_data_set[index][l]
                            print(add_x, end = " ", file = data_file)
                        for l in range(bitlength-1):
                            print(target[l], end = " ", file = data_file)
                        print(target[bitlength-1], file = data_file)
                data_file.close()
            
            

# 5. Prepare test data

In [14]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
import sys
import csv
import copy
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from numpy.linalg import eig
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import pandas as pd
import random


#-------------------------------------------------------------------------
def Sort(sub_list):
 
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of
    # sublist lambda has been used
    return(sorted(sub_list, key = lambda x: x[0]))

#-------------------------------------------------------------------------------
def add_point(normalize_point_list, number_dimension, max_no_point):
    add_normalize_point_list = copy.deepcopy(normalize_point_list)
    number_add_point = max_no_point - len(normalize_point_list)
    size_normalize_point_list = len(normalize_point_list)
    if number_add_point > 0:
        for i in range(number_add_point):
            index = random.randint(0,size_normalize_point_list-1)
            feature = []
            feature = copy.deepcopy(normalize_point_list[index])
            add_normalize_point_list.append(feature)
    return add_normalize_point_list

#---------------------------------------------------------------------#

def prepare_test_data_add_point(input_file_name_ori,max_number_of_cluster,max_no_point,number_dimension,paths):
    total_cluster = max_number_of_cluster

    print("---- read test data and normalize each dimension")
    #bitlength = total_cluster.bit_length()
    bitlength = max_number_of_cluster

    test_data = []
    test_target = []
    temp_target_pattern = []

    input_file_name =input_file_name_ori
    print("\ntest data file:", input_file_name)
    point_list = []
    rows = []


    with open(input_file_name,'r') as data_file:
        for line in data_file:
            # print(line)
            data = line.split(",")
    #         print(data)
            size = len(data)
            if data[0] != '\n':
                cluster_name_bitpattern = []
                for k in range(number_dimension):
    #                 print(data[k])
                    x = float(data[k])
                    # print(type(x))
                    cluster_name_bitpattern.append(x)
                temp_list = copy.deepcopy(cluster_name_bitpattern)    
                point_list.append(temp_list)

    data_file.close()

    # print('cluster_name_bitpattern',cluster_name_bitpattern ) #encoding score 


    normalize_point_list = []
    number_point = len(point_list)
    features = []
    features = copy.deepcopy(point_list)

    dimension_list = []

    for k in range(number_dimension):
        X = []
        for i in range(number_point):
            a = float(features[i][k])
            X.append(a)
        min_xvalue = min(X)
        if min_xvalue < 0:
            for i in range(number_point):
                X[i] = X[i] + abs(min_xvalue)
        max_xvalue = max(X)
        for i in range(number_point):
            X[i] = X[i]/abs(max_xvalue)
        dimension_list.append(X)

    for i in range(number_point):
        features = []
        for k in range(number_dimension):
            a = dimension_list[k][i]
            features.append(a)
        normalize_point_list.append(features)

    add_normalize_point_list = add_point(normalize_point_list, number_dimension, \
                                        max_no_point)

    fout = open(f'{paths}/test-data-normalize.txt', 'w')
    for i in range(len(add_normalize_point_list)):
        for j in range(number_dimension):
            print(add_normalize_point_list[i][j], end = " ", file = fout)
        for j in range(bitlength-1):
            print("0.0", end = " ", file = fout)
        print("0.0", file = fout)
    fout.close()

# 6. Combine all pattern together

In [15]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
import sys
import csv
import copy
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from numpy.linalg import eig
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import pandas as pd
import time
import tensorflow as tf

#-------------------------------------------------------------------------
def Sort(sub_list):
 
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of
    # sublist lambda has been used
    return(sorted(sub_list, key = lambda x: x[0]))

#--------------------------------------------------------------------------
def encode(encode_train_data,paths):
    X = copy.deepcopy(encode_train_data)
    # print(X[0])
    X = np.array(X)
    Y = copy.deepcopy(encode_train_data)
    Y = np.array(Y)
    input_dim = len(encode_train_data[0]) #--- get input dim from 1st pattern ---
    no_input = input_dim
    no_hidden = 313
    no_output = input_dim
    no_epoch = 4000

    print("number of dimensions of encoded vector: ", no_hidden)
    print("number of dimensions of input: ", input_dim)
    print("number of training patterns: ", len(encode_train_data))
    print("number of epochs: ", no_epoch)
    print("in encode")


    model = Sequential()


    model.add(Dense(no_hidden, input_dim = no_input, activation='sigmoid')) 
    model.add(Dense(no_output, activation='sigmoid'))

    model.compile(loss='mse', optimizer='adam', \
                  metrics=[keras.metrics.RootMeanSquaredError()])
#                    metrics=[keras.metrics.BinaryAccuracy()])
    model.fit(X, Y, epochs=no_epoch, verbose = 0)

    ynew = model.predict(X,verbose=0)
    model.save(f'{paths}/model1')


#-------------- use this instructions to get hidden layer value ----------

    get_0_layer_output = K.function([model.layers[0].input],[model.layers[0].output])
    layer_encode_output = get_0_layer_output(X)[0]


    get_1_layer_output = K.function([model.layers[0].input],[model.layers[1].output])
    layer_output = get_1_layer_output(X)[0]


    scores = model.evaluate(X,Y,verbose=0)

    return layer_encode_output, scores[0], scores[1], ynew[0]


def encode_train_test_pattern(start_encode_train_data_set, end_encode_train_data_set,start_cluster,max_number_of_cluster,number_dimension,paths):
    total_cluster = max_number_of_cluster

    print("encode")
    print("number of encode pattern:", start_encode_train_data_set, "-", \
        end_encode_train_data_set )
    #bitlength = total_cluster.bit_length()

        
    output_file_name = f"{paths}/encode-all-cluster.txt"
    print(output_file_name)
    fout = open(output_file_name, "w")
    train_data = []
    target_pattern = []
    temp_target_pattern = []
    
    type_arrangement = 2 #---- 1 = "dist from org", 2 = "arrange_by_dim" ----

    if type_arrangement == 1:
        print("----- sort-by-dist dim: ")
    if type_arrangement == 2:
        print("----- sort-by-dim dim: ")

    input_file_name = f"{paths}/train-cluster{start_cluster}-dataset1.txt"
    with open(input_file_name,'r') as data_file:
        for line in data_file:
            data = line.split()
            size = len(data)
            bitlength = size - number_dimension
            break
    data_file.close()
    print("bitlength", bitlength)

    for j in range(start_cluster,max_number_of_cluster+2):
        for i in range(start_encode_train_data_set, end_encode_train_data_set+1):
            if j == max_number_of_cluster+1:
                input_file_name = f"{paths}/test-data-normalize.txt"
            else:
                input_file_name = f"{paths}/train-cluster" + str(j) + "-dataset" + str(i) + \
                                ".txt"
            point_list = [] 
            with open(input_file_name,'r') as data_file:
                for line in data_file:
                    data = line.split()
                    size = len(data)
                    cluster_name_bitpattern = []
                    cluster_name_bitpattern.clear()
                    for k in range(number_dimension):
                        x = float(data[k])
                        # print(type(x))
                        cluster_name_bitpattern.append(x)
                    temp_target_pattern = []
                    for k in range(bitlength):
                        bitnumber = float(data[k+number_dimension])
                        cluster_name_bitpattern.append(bitnumber)
                        temp_target_pattern.append(bitnumber)
                    temp_list = copy.deepcopy(cluster_name_bitpattern)    
                    point_list.append(temp_list)
            temp = copy.deepcopy(temp_target_pattern)
            target_pattern.append(temp)
        
            data_file.close()
            
            input_from_coordinate = []
            dist_from_org = []
            dist_histogram = []

            if type_arrangement == 1:
                size_point_list = len(point_list)
                for k in range(size_point_list):
                    dist = 0
                    for l in range(number_dimension):
                        dist = dist + point_list[k][l]**2
                    dist = math.sqrt(dist)
                    temp_dist = []
                    temp_dist.append(dist)
                    for l in range(number_dimension):
                        temp_dist.append(point_list[k][l])
                    dist_from_org.append(temp_dist)

                sort_dist_from_org = []
                sort_dist_from_org = Sort(dist_from_org)

                for k in range(size_point_list):
                    for l in range(number_dimension):
                        x = sort_dist_from_org[k][1]
                        # print(x)
                        # print('xxx 250')
                        input_from_coordinate.append(x)

                temp_pattern = copy.deepcopy(input_from_coordinate)
                print(temp_pattern)
                train_data.append(temp_pattern)

            if type_arrangement == 2:
                input_from_coordinate = []
                temp_pattern = []
                size_point_list = len(point_list)
                for l in range(number_dimension):
                    x_dimension_list = []
                    for k in range(size_point_list):
                        x = point_list[k][l]
                        x_dimension_list.append(x)
                    x_dimension_list.sort()
                    input_from_coordinate = input_from_coordinate + x_dimension_list
                # print(input_from_coordinate)     
                temp_pattern = copy.deepcopy(input_from_coordinate)
                train_data.append(temp_pattern)
            if j == max_number_of_cluster+1:
                break

    print(type(train_data))
    train_data = np.array(train_data)

    do_encode =0 
    
    if  do_encode ==0:
        encode_output = copy.deepcopy(train_data)
    if  do_encode ==1:
        encode_output, loss, error, output_y = encode(train_data,paths)

        print("-------- encode output")
        print("loss: ", loss)
        print("rms error", error)
    no_of_encoded_pattern = len(encode_output)
    dim_encode_pattern = len(encode_output[0]) #--- get dim of each ecodeed pattern ---
    print("number of encoded patterns:", no_of_encoded_pattern)

    for i in range(no_of_encoded_pattern):
        cluster_no = i // end_encode_train_data_set + 1
        dataset_no = i % end_encode_train_data_set + 1
        print(cluster_no, " ", dataset_no, " ", end = " ", file = fout)
        for k in range(dim_encode_pattern):
            print(encode_output[i][k], end = " ", file = fout)
        for l in range(bitlength-1):
            print(target_pattern[i][l], end = " ", file = fout)
        print(target_pattern[i][bitlength-1], file = fout)


    fout.close()

# 8. Split Train and Test

In [16]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
# from keras.layers.core import Dense
from keras.layers import Dense
from keras import backend as K
from os.path import exists
from pathlib import Path
from os.path import exists as file_exists
import sys
from sklearn.model_selection import train_test_split
import copy
import time
import random
import matplotlib.pyplot as plt
import itertools
import psutil
from sklearn.metrics import silhouette_score, davies_bouldin_score, accuracy_score, homogeneity_score
from sklearn.metrics import completeness_score
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import v_measure_score

from sklearn.metrics import confusion_matrix
#---------------------------------------------------------------------------

def adjust_predict_output(predict_output_test, test_target, print_adjust = 'no', \
                          print_wrong_prediction = 'no', target_defined = 'yes'):
    dim = len(predict_output_test[0])
    number_test_pattern = len(predict_output_test)
    all_adjust_predict = []

#--------- find max value, reset max value to 0.5 and the rest to 0.0 -------  
    
    for i in range(number_test_pattern):
        adjust_predict = []
        max_value = predict_output_test[i][0]
        for j in range(dim):
            if max_value < predict_output_test[i][j]:
                max_value = predict_output_test[i][j]

        for j in range(dim):
            if predict_output_test[i][j] < max_value:
                adjust_predict.append(0.0)
            else:
                adjust_predict.append(0.5)
        temp_list = copy.deepcopy(adjust_predict)
        all_adjust_predict.append(temp_list)
        
    number_test_set = len(test_target)
    # if print_adjust == 'yes':
    #     for i in range(number_test_set):
    #         print("\nPredicted = %s" % (predict_output_test[i]), "\ntarget = %s" % \
    #               (test_target[i]))
    #         print("adjust Predicted = %s" % (all_adjust_predict[i]), "\ntarget = %s" % \
    #               (test_target[i]))
    #         print(" ")

    dim = len(test_target[0])
    count_same_answer = 0
    wrong_predict = []

#------- compute accuracy of each cluster and total accuracy ---------------- 

    pattern_cluster = []
    accuracy_cluster = []
    
    for i in range(dim):
        pattern_cluster.append(0)
        accuracy_cluster.append(0)

    if target_defined == 'yes':   
        for i in range(number_test_set):
            for k in range(dim):
                if test_target[i][k] == 0.5: #------ cannot use if test target = 0.0
                    cluster_index = k
                    break
            pattern_cluster[cluster_index] = pattern_cluster[cluster_index] + 1
            print(pattern_cluster)

        no_under_predict = 0
        no_over_predict = 0

        for i in range(number_test_set):
            count = 0
            for k in range(dim):
                if test_target[i][k] == 0.5:
                    cluster_index = k
                    break

            for k in range(dim):
                if all_adjust_predict[i][k] == 0.5:
                    predict_index = k
                    break
            if predict_index < cluster_index:
                no_under_predict = no_under_predict + 1
            if predict_index > cluster_index:
                no_over_predict = no_over_predict + 1
            
            for j in range(dim):
                if all_adjust_predict[i][j] == test_target[i][j]:
                    count = count + 1
            if count == dim:
                count_same_answer = count_same_answer + 1
                accuracy_cluster[cluster_index] = accuracy_cluster[cluster_index] + 1
            else:
                wrong_predict.append(i)
                
    
        adjust_accuracy = 100*count_same_answer/number_test_set

        for i in range(dim):
            if pattern_cluster[i] > 0:
                accuracy_cluster[i] = accuracy_cluster[i]*100/pattern_cluster[i]
            else:
                accuracy_cluster[i] = -1
        
        if print_wrong_prediction == 'yes':
            size = len(wrong_predict)
            print("----- wrong prediction")
            for i in range(size):
                index = wrong_predict[i]
                print("\Predicted = %s\n" % (predict_output_test[index]), "target = %s\n" % \
                      (test_target[index]))

        return all_adjust_predict,adjust_accuracy, accuracy_cluster, no_under_predict, no_over_predict

    if target_defined == 'no':
        adjust_accuracy = -1
        no_under_predict = -1
        no_over_predict = -1
        return all_adjust_predict,adjust_accuracy, accuracy_cluster, no_under_predict, no_over_predict

# #------------------------------------------------------------------------------
def separate_specific_one_test_data(all_train_data, all_target_data, \
                                    test_cluster_no, test_dataset_no):
    
    size_train_data = len(all_train_data)
    temp_all_train_data = copy.deepcopy(all_train_data)
    temp_all_train_target = copy.deepcopy(all_target_data)
    
    X_all_train_data = []
    X_all_train_target = []
    X_all_test_data = []
    X_all_test_target = []
    
    for i in range(size_train_data):
        cluster_no = temp_all_train_data[i][0]
        dataset_no = temp_all_train_data[i][1]
        if test_cluster_no == cluster_no and test_dataset_no == dataset_no:
            X_all_test_data.append(temp_all_train_data[i])
            X_all_test_target.append(temp_all_train_target[i])
        else:
            X_all_train_data.append(temp_all_train_data[i])
            X_all_train_target.append(temp_all_train_target[i])
        

    return X_all_train_data, X_all_test_data, X_all_train_target, \
           X_all_test_target

In [17]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LogisticRegression
import numpy as np
import sys
import csv
import copy
import random
import ast
from sklearn.multioutput import RegressorChain
from sklearn.svm import SVR
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import time

import time as tm
from resource import *

def train_test(max_number_of_cluster,paths,number_dimension,real_n,min_rp,max_rp,fold_num,C_x):
    print("read training data")
    train_data = []
    train_feature_vector = []
    train_target = []
    #bitlength = total_cluster.bit_length()
    bitlength = max_number_of_cluster
    all_train_data = []
    all_target_data = []
    all_train_result=[]
    predicted_class=[]
    prediction_times = []  # List to store prediction times for each fold
    # Lists to store performance metrics over 100 repetitions
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    mcc_scores = [] 

    input_file_name = f"{paths}/encode-all-cluster.txt"
    print(input_file_name)

    count = -1
    with open(input_file_name,'r') as train_data_file:   
        for line in train_data_file:
            data = line.split()
            dim = len(data)
            inputdim = dim - bitlength - 2 #-- 2: cluster_no and dataset_no ---
            train_feature_vector = []
            cluster_name_bitpattern = []
            cluster_no = int(data[0])
            dataset_no = int(data[1])
            train_feature_vector.append(cluster_no)
            train_feature_vector.append(dataset_no)
            for j in range(2,inputdim+2):
                coordinate = float(data[j])
                train_feature_vector.append(coordinate)
            temp1 = copy.deepcopy(train_feature_vector)
            train_data.append(temp1)
            all_train_data.append(temp1)

            for k in range(bitlength):
                bitnumber = float(data[k+inputdim+2])
                cluster_name_bitpattern.append(bitnumber)
            temp2 = copy.deepcopy(cluster_name_bitpattern)
            train_target.append(temp2) #----- modify --------
            all_target_data.append(temp2)
                
    train_data_file.close()

    #-------------------------------------------------------#
    # Select test 100 
    no_test_pattern = 1
    no_train_pattern = len(all_train_data)
    test_percent = no_test_pattern/no_train_pattern
    train_percent = 1.0 - test_percent
    average_accuracy = 0
    print("train : test = ", train_percent*100, ":", test_percent*100)

    #####################################################################
    ################ Assign the number of Fold 
    #####################################################################

    fold_number = fold_num
    print("----- start training and testing: ", fold_number, " fold")

    all_adjust_predict_fold=[]
    fold_sol=[]
    fold_clus=[]
    modified_values_test=[]
    predicted_class=[]
    
    
    start=1
    
    print('All train:',len(all_train_data))
    time_per_fold = []
    memory_per_fold = []

    for k in range(start,fold_number+1):
        print("\n\n=======================================================")
        fout = open(f'{paths}/predict-number-cluster-result_{start}_{C_x}.txt', 'a')  # Open file in append mode
        fout.write(f"----- fold {k} -----\n")
        print(f"-------------- fold no. : {k}")
        print("number dimension: ", number_dimension)

        test_set_separate_type = 1
    #------------- get encoded iris data in encode-all-cluster file ------- 
        if test_set_separate_type == 1:
            test_cluster_no = real_n  #---- 6
            # test_dataset_no = random.randint(min_rp,max_rp) #---- 1
            test_dataset_no=min_rp+k-1
            print(f"Processing dataset number: {test_dataset_no}")

            print("----- test specific pattern in encode test data")
            print("cluster no:", test_cluster_no, "dataset no:", test_dataset_no)
            
            Xtrain_input, Xtest_input, Xtrain_target, Xtest_target = \
                separate_specific_one_test_data(all_train_data, all_target_data, \
                                            test_cluster_no, test_dataset_no)
            size_Xtrain_input = len(Xtrain_input)
            size_Xtest_input = len(Xtest_input)
            
            # print('size_Xtrain_input',size_Xtrain_input)
            # print('size_Xtest_input',size_Xtest_input )
           
                
        if test_set_separate_type == 2: #--- delete cluster 6 dataset 1 -----
            Xtrain_input = []
            Xtrain_target = []
            Xtest_input = []
            Xtest_target = []
            state_value = random.randint(1,100)

        
            train_input, test_input, train_target, test_target = train_test_split(\
                    all_train_data, all_target_data, train_size = train_percent,\
                    test_size = test_percent, random_state = state_value, \
                    shuffle = True) 

            Xtrain_input = copy.deepcopy(train_input)
            Xtrain_target = copy.deepcopy(train_target)
            Xtest_input = copy.deepcopy(test_input)
            Xtest_target = copy.deepcopy(test_target)
        
            size_Xtrain_input = len(Xtrain_input)
            size_Xtest_input = len(Xtest_input)
        
        test_cluster_dataset_list = []
        for j in range(size_Xtest_input):
            test_cluster_dataset_list.append([Xtest_input[j][0],Xtest_input[j][1]])
            
        for j in range(size_Xtrain_input):
            del Xtrain_input[j][0]
            del Xtrain_input[j][0]

        for j in range(size_Xtest_input):
            del Xtest_input[j][0] #---- remove cluster no. ----------
            del Xtest_input[j][0]

     
        #### Train Test Model        
        print("regressorchian: multi.fit(Xtrain_input,Xtrain_target)")
        
        
        multi = RegressorChain(SVR(kernel='rbf', C=C_x, tol=0.001, gamma='scale', epsilon=0.1)) #defalut

        multi.fit(Xtrain_input, Xtrain_target)
        
       
        # Record time before predictions
        start_time = time.time()
        now = time.asctime()
        time_stamp = now.split(' ')
        hour = now.split(' ')[3]
        time_stamp[3] = '-'.join(hour.split(':'))

        time_stamp = '_'.join(time_stamp[1:-1])
        # start_time = tm.time()
        Xpredict_test = multi.predict(Xtest_input)      
       
        running_info = getrusage(RUSAGE_SELF)
        _time = (time.time() - start_time) #running_info.ru_utime + running_info.ru_stime
        hour = _time // 3600
        minutes = (_time  - (3600 * hour)) // 60
        seconds = _time - (hour * 3600) - (minutes * 60)
        memory = (running_info.ru_maxrss/1e6)
        timess=[hour,minutes,seconds]
        time_per_fold.append(_time)
        memory_per_fold.append(memory)
        print(f'Summary_Files: {int(hour)}:{int(minutes)}:{round(seconds)} (hh:mm:ss) and {memory} (GB)')
    
        print(f'Fold {k}: {int(hour)}:{int(minutes)}:{round(seconds)} (hh:mm:ss) and {memory} (GB)')
        
            
        #### Calculate performance
        # Count the occurrences of 0.5 in each column
        all_adjust_predict, adjust_accuracy, accuracy_cluster, no_under_predict, no_over_predict = \
        adjust_predict_output(Xpredict_test, Xtest_target, print_adjust='yes', print_wrong_prediction='yes', target_defined='no')
        y_pred = np.array(all_adjust_predict[0])
        y_test = np.array(Xtest_target[0])
        # Convert lists to NumPy arrays
        y_pred = np.argmax(y_pred, axis=1) if y_pred.ndim ==0.5 else np.ceil(y_pred).astype(int)
        y_test= np.argmax(y_test, axis=1) if y_test.ndim ==0.5 else np.ceil(y_test).astype(int)

        print('y_pred',y_pred)   
        print('y_test',y_test)    
        
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        recalls.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, average='macro', zero_division=0))
        mcc_scores.append(matthews_corrcoef(y_test, y_pred))
        

        transposed_data = np.array(all_adjust_predict).T

        # Count the occurrences of 0.5 in each column
        counts_0_5 = [list(column).count(0.5) for column in transposed_data]

        # Find the column index with the maximum count of 0.5
        max_count_index = counts_0_5.index(max(counts_0_5))
        print("Column with the maximum count of 0.5:", max_count_index)
        # Print the index of the column with the maximum count of 0.5
    
        
        fold_sol.append(counts_0_5)
        fold_clus.append(max_count_index+1)
        predicted_class.append(max_count_index + 1)
        print(fold_clus)
        # print("Indices of 0.5:", counts_0_5 )
        print(f"Fold_{k}_predicted: the number of class is {max_count_index + 1}", end=" ")

    #----------------------------------------- use the original labels
        print(" ")
        print("total adjust accuracy:", adjust_accuracy, "%\n")
        adjust_accuracy
        dim = len(Xtest_target[0])
        for i in range(dim):
            print("cluster ", i+1, " accuracy: ", accuracy_cluster[i], "%")
            if accuracy_cluster[i] > -1:
                accuracy = accuracy_cluster[i]
                no_cluster = i+1
  
        fout.write(f"The predicted number from fold {k} : {max_count_index + 1}\n")
    fout.write(f"The predicted number from fold 1-{k} : {predicted_class}\n")
   
    counter = Counter(predicted_class)
    most_common = counter.most_common(1)[0]  # Get the most common element and its count

    most_frequent_number = most_common[0]
    frequency = most_common[1]
    acc = (frequency/len(predicted_class))*100

    print('Times',time_per_fold)
    print('Memory',memory_per_fold)
    
    print('\n')
    print('Times',sum(time_per_fold))
    print('Memory',np.average(memory_per_fold))
    
    print(f"The most frequent number is: {most_frequent_number} with a count of: {frequency}")
    print(counter)
    fout.write(f"All results : {counter}\n")
    fout.write(f"The most predicted number from fold 1-{k} : {most_frequent_number} with a count of: {frequency}\n")
    fout.write(f"Accuracy of {most_frequent_number}: {acc}\n")
    fout.write(f"The most predicted number from fold 1-{k} : {most_frequent_number} with a count of: {frequency}\n")
    fout.write(f"Accuracy of {most_frequent_number}: {acc}\n")
       
    fout.write(f"Times 1-{k} : {time_per_fold}\n")
    fout.write(f"Times 1-{k} total: {sum(time_per_fold)}\n")
    fout.write(f"Memory_per_fold 1-{k} : {memory_per_fold}\n")
    fout.write(f"Memory_per_fold 1-{k} average: {np.average(memory_per_fold)}\n")
    ######################### 
    
    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1_score = np.mean(f1_scores)
    std_f1_score = np.std(f1_scores)

    mean_mcc = np.mean(mcc_scores)
    std_mcc = np.std(mcc_scores)
    
    fout.write(f"\n")
    fout.write(f"Summary Results: For folds 1-{fold_num}\n")
    fout.write(f"mean_accuracy : {mean_accuracy}\n")
    fout.write(f"std_accuracy  : {std_accuracy}\n")
    fout.write(f"mean_precision: {mean_precision}\n")
    fout.write(f"std_precision : {std_precision }\n")
    fout.write(f"mean_recall   : {mean_recall}\n")
    fout.write(f"std_recall    : {std_recall }\n")
    fout.write(f"mean_f1_score : {mean_f1_score}\n")
    fout.write(f"std_f1_score  : {std_f1_score}\n")
    fout.write(f"mean_mcc : {mean_mcc}\n")
    fout.write(f"std_mcc  : {std_mcc}\n")
    fout.close()
 
    

In [18]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor

def train_test_train(max_number_of_cluster,paths,number_dimension,real_n,min_rp,max_rp,fold_num,C_x):
    print("read training data")
    train_data = []
    train_feature_vector = []
    train_target = []
    #bitlength = total_cluster.bit_length()
    bitlength = max_number_of_cluster
    all_train_data = []
    all_target_data = []
    all_train_result=[]
    predicted_class=[]
    prediction_times = []  # List to store prediction times for each fold
    # Lists to store performance metrics over 100 repetitions
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    mcc_scores = [] 

  

    input_file_name = f"{paths}/encode-all-cluster.txt"
    print(input_file_name)

    count = -1
    with open(input_file_name,'r') as train_data_file:   
        for line in train_data_file:
            data = line.split()
            dim = len(data)
            inputdim = dim - bitlength - 2 #-- 2: cluster_no and dataset_no ---
            train_feature_vector = []
            cluster_name_bitpattern = []
            cluster_no = int(data[0])
            dataset_no = int(data[1])
            train_feature_vector.append(cluster_no)
            train_feature_vector.append(dataset_no)
            for j in range(2,inputdim+2):
                coordinate = float(data[j])
                train_feature_vector.append(coordinate)
            temp1 = copy.deepcopy(train_feature_vector)
            train_data.append(temp1)
            all_train_data.append(temp1)

            for k in range(bitlength):
                bitnumber = float(data[k+inputdim+2])
                cluster_name_bitpattern.append(bitnumber)
            temp2 = copy.deepcopy(cluster_name_bitpattern)
            train_target.append(temp2) #----- modify --------
            all_target_data.append(temp2)
                
    train_data_file.close()

    #-------------------------------------------------------#
    no_test_pattern = 1
    no_train_pattern = len(all_train_data)
    test_percent = no_test_pattern/no_train_pattern
    train_percent = 1.0 - test_percent
    average_accuracy = 0
    print("train : test = ", train_percent*100, ":", test_percent*100)

    #####################################################################
    ################ Assign the number of Fold 
    #####################################################################

    # fold_number = int(fold_num/20)
    fold_number = 3
    print("----- start training and testing: ", fold_number, " fold")

    all_adjust_predict_fold=[]
    fold_sol=[]
    fold_clus=[]
    modified_values_test=[]
    predicted_class=[]
    count_pre_list=[]
    
    start=1
    count_pre=0
    fold_number=fold_number
    
    
    print('All train:',len(all_train_data))
    time_per_fold = []
    memory_per_fold = []

    for k in range(start,fold_number+1):
        test_dataset_no=min_rp+k-1
        print("\n\n=======================================================")
        fout = open(f'{paths}/predict-number-cluster-result_{start}_Train_{C_x}.txt', 'a')  # Open file in append mode
        fout.write(f"----- fold {k} -----\n")
        print(f"-------------- fold no. : {k}")
        print("number dimension: ", number_dimension)

        test_set_separate_type = 1
    #------------- get encoded iris data in encode-all-cluster file ------- 
        if test_set_separate_type == 1:
            test_cluster_no = real_n  #---- 6
            # test_dataset_no = random.randint(min_rp,max_rp) #---- 1
            # for test_dataset_no in range(min_rp, max_rp + 1):  # Loop through all values from min_rp to max_rp (inclusive)
            test_dataset_no=min_rp+k-1
            print(f"Processing dataset number: {test_dataset_no}")

            print("----- test specific pattern in encode test data")
            print("cluster no:", test_cluster_no, "dataset no:", test_dataset_no)
            
            Xtrain_input, Xtest_input, Xtrain_target, Xtest_target = \
                separate_specific_one_test_data(all_train_data, all_target_data, \
                                            test_cluster_no, test_dataset_no)
            size_Xtrain_input = len(Xtrain_input)
            size_Xtest_input = len(Xtest_input)
            
            # print('size_Xtrain_input',size_Xtrain_input)
            # print('size_Xtest_input',size_Xtest_input )
           
                
        if test_set_separate_type == 2: #--- delete cluster 6 dataset 1 -----
            Xtrain_input = []
            Xtrain_target = []
            Xtest_input = []
            Xtest_target = []
            state_value = random.randint(1,100)

        
            train_input, test_input, train_target, test_target = train_test_split(\
                    all_train_data, all_target_data, train_size = train_percent,\
                    test_size = test_percent, random_state = state_value, \
                    shuffle = True) 

            Xtrain_input = copy.deepcopy(train_input)
            Xtrain_target = copy.deepcopy(train_target)
            Xtest_input = copy.deepcopy(test_input)
            Xtest_target = copy.deepcopy(test_target)
        
            size_Xtrain_input = len(Xtrain_input)
            size_Xtest_input = len(Xtest_input)
        
        test_cluster_dataset_list = []
        for j in range(size_Xtest_input):
            test_cluster_dataset_list.append([Xtest_input[j][0],Xtest_input[j][1]])
            
        for j in range(size_Xtrain_input):
            del Xtrain_input[j][0]
            del Xtrain_input[j][0]

        for j in range(size_Xtest_input):
            del Xtest_input[j][0] #---- remove cluster no. ----------
            del Xtest_input[j][0]

        
        # no_hidden_neuron = 211        
        #   # Record start time for prediction
        # # start_time = tm.time()
        # Xpredict_output_test,all_adjust_predict,accuracy_cluster, no_under_predict, no_over_predict=\
        # train_network(Xtrain_input, Xtrain_target, Xtest_input, Xtest_target,k,no_hidden_neuron,max_number_of_cluster,paths)
        
        #### Train Test Model        
        print("regressorchian: multi.fit(Xtrain_input,Xtrain_target)")
        
        # order_permutation = [3, 2, 1, 0]                   
        multi = RegressorChain(SVR(kernel='rbf', C=C_x, tol=0.001, gamma='scale', epsilon=0.1)) #defalut
        # multi = MLPRegressor(activation='relu', hidden_layer_sizes=(1000,1000), alpha=0.001, 
        #            shuffle=True, max_iter=5000)
        multi.fit(Xtrain_input, Xtrain_target)       





    #    ############################################# Check 100% accuracy?
    #     # Make predictions on the training data
    #     Xpredict_train = multi.predict(Xtrain_input)

     
    #     all_adjust_predict, adjust_accuracy, accuracy_cluster, no_under_predict, no_over_predict = \
    #     adjust_predict_output(Xpredict_train, Xtrain_target, print_adjust='yes', print_wrong_prediction='yes', target_defined='no')
    #     X_pred_train = np.array(all_adjust_predict[0])
    #     X_train_tar = np.array(Xtrain_target[0])
    #     y_pred_train = np.argmax(X_pred_train, axis=1) if X_pred_train.ndim ==0.5 else np.ceil(X_pred_train).astype(int)
    #     y_test_train= np.argmax( X_train_tar, axis=1) if  X_train_tar.ndim ==0.5 else np.ceil(X_train_tar).astype(int)
        
    #     # Compare predicted values to actual target values
    #     print('X_pred_train',y_pred_train )
    #     print(' X_train_tar ',  y_test_train)
        
    #     train_accuracy = accuracy_score(X_train_tar , y_pred_train )

    #     # Print the accuracy
    #     print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    #     print(f"Training accuracy: {train_accuracy * 100:.2f}%")
    #     print( )
        
        # # Check if training accuracy is 100%
        # if train_accuracy == 1.0:
        #     print("The model has achieved 100% accuracy on the training data.")
        # else:
        #     print("The model has not achieved 100% accuracy on the training data.")

     
        # Record time before predictions
        start_time = time.time()
        now = time.asctime()
        time_stamp = now.split(' ')
        hour = now.split(' ')[3]
        time_stamp[3] = '-'.join(hour.split(':'))

        time_stamp = '_'.join(time_stamp[1:-1])
        # start_time = tm.time()
        ##################################################################################################################################
        # Xpredict_test = multi.predict(Xtest_input)
        Xpredict_test = multi.predict(Xtrain_input)
        
        # print(Xpredict_test)
        # print()
        # print(Xtrain_target)

        # Record time after predictions
     
        # end_time = tm.time()
        
        # prediction_time = end_time - start_time
        # print(f"Prediction time: {prediction_time:.2f} seconds")
        
        # print("\nXpredict test")
        # print(Xpredict_test)
        # print("\nXtest target")
        # print(test_target)        
       
        running_info = getrusage(RUSAGE_SELF)
        _time = (time.time() - start_time) #running_info.ru_utime + running_info.ru_stime
        hour = _time // 3600
        minutes = (_time  - (3600 * hour)) // 60
        seconds = _time - (hour * 3600) - (minutes * 60)
        memory = (running_info.ru_maxrss/1e6)
        timess=[hour,minutes,seconds]
        time_per_fold.append(_time)
        memory_per_fold.append(memory)
        print(f'Summary_Files: {int(hour)}:{int(minutes)}:{round(seconds)} (hh:mm:ss) and {memory} (GB)')
    
        print(f'Fold {k}: {int(hour)}:{int(minutes)}:{round(seconds)} (hh:mm:ss) and {memory} (GB)')
        
        Xtest_target = Xtrain_target
        #### Calculate performance
        # Count the occurrences of 0.5 in each column
        all_adjust_predict, adjust_accuracy, accuracy_cluster, no_under_predict, no_over_predict = \
        adjust_predict_output(Xpredict_test, Xtest_target, print_adjust='yes', print_wrong_prediction='yes', target_defined='no')
        y_pred = np.array(all_adjust_predict)
        y_test = np.array(Xtest_target)
        # y_pred = np.array(all_adjust_predict[0])
        # y_test = np.array(Xtest_target[0])
        # Convert lists to NumPy arrays
        y_pred = np.argmax(y_pred, axis=1) if y_pred.ndim ==0.5 else np.ceil(y_pred).astype(int)
        y_test= np.argmax(y_test, axis=1) if y_test.ndim ==0.5 else np.ceil(y_test).astype(int)

        # print('y_pred: ',y_pred)   
        # print('y_test: ',y_test)   
        
        # Check if the shapes match
        if y_pred.shape == y_test.shape:
            # Compare element-wise
            comparison = np.isclose(y_pred, y_test)
            
            # Check each row (sublist) for all True
            match_counts = [1 if np.all(row) else 0 for row in comparison]

            print("Element-wise comparison:\n", comparison)
            print("Match count for each sublist (1 if all True, 0 otherwise):", match_counts)
        else:
            print("The shapes of y_pred and y_test do not match.")

        train_accuracy = (np.sum(match_counts) / y_test.shape[0]) * 100
        print('Sum',np.sum(match_counts))
        print('Y-test_shape',y_test.shape[0])
        
        # Print train accuracy with two decimal places
        print('Train accuracy: {:.2f}%'.format(train_accuracy))

        # Append the rounded train accuracy to the list
        count_pre_list.append(round(train_accuracy, 2))
        
        # accuracies.append(accuracy_score(y_test, y_pred))
        # precisions.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        # recalls.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        # f1_scores.append(f1_score(y_test, y_pred, average='macro', zero_division=0))
        # mcc_scores.append(matthews_corrcoef(y_test, y_pred))
        


        # Transpose the data to get columns
        transposed_data = np.array(all_adjust_predict).T

        # Count the occurrences of 0.5 in each column
        counts_0_5 = [list(column).count(0.5) for column in transposed_data]

        # Find the column index with the maximum count of 0.5
        max_count_index = counts_0_5.index(max(counts_0_5))
        print("Column with the maximum count of 0.5:", max_count_index)
        # Print the index of the column with the maximum count of 0.5
    
        
        fold_sol.append(counts_0_5)
        fold_clus.append(max_count_index+1)
        predicted_class.append(max_count_index + 1)
        print(fold_clus)
        # print("Indices of 0.5:", counts_0_5 )
        print(f"Fold_{k}_predicted: the number of class is {max_count_index + 1}", end=" ")

    #----------------------------------------- use the original labels
        print(" ")
        print("total adjust accuracy:", adjust_accuracy, "%\n")
        adjust_accuracy
        dim = len(Xtest_target[0])
        for i in range(dim):
            print("cluster ", i+1, " accuracy: ", accuracy_cluster[i], "%")
            if accuracy_cluster[i] > -1:
                accuracy = accuracy_cluster[i]
                no_cluster = i+1
        # print("fold:", k, "cluster:", cluster_no, " dataset:", dataset_no,\
        #     "accuracy:", accuracy)
        # print("number of under-predicted patterns: ", no_under_predict)
        # print("number of over-predicted patterns: ", no_over_predict)
        
        fout.write(f"The predicted number from fold {k} : {max_count_index + 1}\n")
    fout.write(f"The predicted number from fold 1-{k} : {predicted_class}\n")
   
    counter = Counter(predicted_class)
    most_common = counter.most_common(1)[0]  # Get the most common element and its count

    most_frequent_number = most_common[0]
    frequency = most_common[1]
    acc = (frequency/len(predicted_class))*100

    print('Times',time_per_fold)
    print('Memory',memory_per_fold)
    
    print('\n')
    print('Times',sum(time_per_fold))
    print('Memory',np.average(memory_per_fold))
    
    print(f"The most frequent number is: {most_frequent_number} with a count of: {frequency}")
    print(counter)
    fout.write(f"All results : {counter}\n")
    fout.write(f"The most predicted number from fold 1-{k} : {most_frequent_number} with a count of: {frequency}\n")
    fout.write(f"Accuracy of {most_frequent_number}: {acc}\n")
    fout.write(f"The most predicted number from fold 1-{k} : {most_frequent_number} with a count of: {frequency}\n")
    fout.write(f"Accuracy of {most_frequent_number}: {acc}\n")
       
    fout.write(f"Times 1-{k} : {time_per_fold}\n")
    fout.write(f"Times 1-{k} total: {sum(time_per_fold)}\n")
    fout.write(f"Memory_per_fold 1-{k} : {memory_per_fold}\n")
    fout.write(f"Memory_per_fold 1-{k} average: {np.average(memory_per_fold)}\n")
    ######################### 
    
    print('count_pre_list',count_pre_list) 
    print('accuracy_mean',np.mean(count_pre_list) )
    fout.write(f"accuracy_all  : {count_pre_list}\n")
    fout.write(f"accuracy_mean  : {np.mean(count_pre_list)}\n")
    fout.close()
     
    

In [19]:
kmer = 6

n_components=5

number_dimension= n_components


n_real= 3


num_p= 400

num_real = 300

start_gen_gaussian_train_data_set = 1
end_gen_gaussian_train_data_set   = num_p

start_gen_real_train_data_set = num_p-num_real +1
end_gen_real_train_data_set  = num_p


start_cluster           = 1
max_number_of_cluster   = 10

start_add_point_train_data_set = 1
end_add_point_train_data_set   = num_p

start_encode_train_data_set  = 1
end_encode_train_data_set    = num_p


increase_r = 3

fold_num = 100

C_x=1.0

path='/home/admin1/Program5_Test2024/iDeLUCS-master/Example'



In [20]:

name='mtInsects'
name='balanced_mtInsects'
# name='Cyprinidae'

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import argparse
import sys
import time

import csv
from resource import *

start_time = time.time()
now = time.asctime()
time_stamp = now.split(' ')
hour = now.split(' ')[3]
time_stamp[3] = '-'.join(hour.split(':'))

time_stamp = '_'.join(time_stamp[1:-1])

#1 Create new folder and Read files 
folder_name = os.getcwd()
folder_name = f'{folder_name}/Results_{name}_{C_x}'

if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

results_folder = f'{folder_name}'
paths= results_folder

if not os.path.isdir(f'{results_folder}'):
    os.mkdir(f'{results_folder}')
        

In [21]:
# name='Fungi'
name='balanced_Fungi'
# name='Cyprinidae'
# name='Cyprinidae'
# name='mtInsects'
# name='balanced_mtInsects'
# name='Protists'
# name='Actinopterygii'

# name='balanced_Protists'
# name='Dengue'
# name='HBV'
# name='Dengue'
# name='mtInsects'
# name='balanced_mtInsects'
# # name='Neopterygii'
# name='Vertebrata'

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import argparse
import sys
import time

import csv
from resource import *

start_time = time.time()
now = time.asctime()
time_stamp = now.split(' ')
hour = now.split(' ')[3]
time_stamp[3] = '-'.join(hour.split(':'))

time_stamp = '_'.join(time_stamp[1:-1])

#1 Create new folder and Read files 
folder_name = os.getcwd()
folder_name = f'{folder_name}/Results_{name}_{C_x}'

if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

results_folder = f'{folder_name}'
paths= results_folder

if not os.path.isdir(f'{results_folder}'):
    os.mkdir(f'{results_folder}')
        
results_folder = f'{folder_name}'
paths= results_folder

if not os.path.isdir(f'{results_folder}'):
    os.mkdir(f'{results_folder}')
        

fasta_file = f"{path}/{name}.fas"
GT_file = f"{path}/{name}_GT.tsv"

names, lengths, ground_truth, cluster_dis, class_ranges = SummaryFasta(fasta_file, GT_file=GT_file)
print("The cluster distribution is:")
print (cluster_dis)
min_size = min(cluster_dis.values())
max_size = max(cluster_dis.values())

# From cluster_dis can get min max cluster size
min_clusters = [k for k, v in cluster_dis.items() if v == min_size]
max_clusters = [k for k, v in cluster_dis.items() if v == max_size]


stats = {"n_seq": len(lengths),
        "min_len": np.min(lengths),
        "max_len": np.max(lengths),
        "avg_len": np.mean(lengths)}

print( f'No. Sequences: \t {stats["n_seq"]:,}')
print(f'Min. Length: \t {stats["min_len"]:,}')
print(f'Max. Length: \t {stats["max_len"]:,}')
print(f'Avg. Length: \t {round(stats["avg_len"],2):,}')
print(f"Min cluster size: {min_size} ({', '.join(min_clusters)})")
print(f"Max cluster size: {max_size} ({', '.join(max_clusters)})")

# Save sequence statistics to a text file
result_file = f"{name}_sequence_statistics_and_cluster_distribution.txt"
with open(result_file, 'w') as f:
    f.write("Cluster Distribution:\n")
    for cluster, count in cluster_dis.items():
        f.write(f'{cluster}: {count}\n')

    f.write("\nSequence Statistics:\n")
    f.write(f'No. Sequences: \t {stats["n_seq"]:,}\n')
    f.write(f'Min. Length: \t {stats["min_len"]:,}\n')
    f.write(f'Max. Length: \t {stats["max_len"]:,}\n')
    f.write(f'Avg. Length: \t {round(stats["avg_len"], 2):,}\n')
    f.write(f'Class. Range: \t {class_ranges}\n')
    
unique = sorted(list(set(ground_truth)))
GT = list(map(lambda x: unique.index(x), ground_truth))
print(GT)

#2 Feature Extraction 
# Read the fasta sequences
fastas = read_nucleotide_sequences(fasta_file)
# Generate Kmer encodings
encodings = Kmer(fastas, k=kmer, type='DNA', upto=False, normalize=True)
# Save the encodings to a file
output = 'encoding.txt'
save_file(encodings, 'svm', output)
# Preprocess the saved file
with open(output, 'r') as f:
    values_list = preprocess(f)
# Define the output file path
output_file = f'{name}_Kmer_{kmer}.txt'
# Preprocess the saved file
with open(output, 'r') as f:
    values_list = preprocess(f)
# Write values_list to the file with comma delimiter
np.savetxt(output_file, values_list, delimiter=',')
os.remove(output)

#3 PCA 
X_scale, X_scale_pca, pca_components_,exp_var_pca,total_explained_variance,pca =run_pca(values_list,n_components)
X_scale_pca
df_pca = pd.DataFrame( X_scale_pca[:, :n_components], columns=[f'PC{i+1}' for i in range(n_components)])
df_pca['Label'] = GT

#4 Normalized to 0-10
scaler = MinMaxScaler(feature_range=(0, 10))

# Fit and transform the data
X_normalized = scaler.fit_transform(X_scale_pca)
countx=X_normalized.shape[0]
print(countx)

top_components =n_components
df_pca_norm= pd.DataFrame(X_normalized[:, :top_components], columns=[f'PC{i+1}' for i in range(top_components)])
df_pca_norm['Label'] = GT

# feature of real data in the X_normalized
with open(f"{paths}/preprocessed_data.txt", "w") as f:
    np.savetxt(f,X_normalized, delimiter=",")

# 5 Generate Guassian-only
Generate_guassian_make_bobs(start_gen_gaussian_train_data_set,end_gen_gaussian_train_data_set,num_real,start_cluster,max_number_of_cluster,min_size,max_size, number_dimension,0.1,1,n_real,paths)
# print(" ")

# 6 Generate KDE-only
Generate_real(start_gen_real_train_data_set,end_gen_real_train_data_set,X_normalized, class_ranges, n_real,GT,increase_r,max_number_of_cluster,paths)

# 7 add point of each pretrain  equal to the number of points of real dataset 
train_only_add_points_before_encode_data(start_add_point_train_data_set, end_add_point_train_data_set,start_cluster,max_number_of_cluster,countx,number_dimension,paths)

# # 8 add real data set and normalized into 0-1 
prepare_test_data_add_point(f"{paths}/preprocessed_data.txt",max_number_of_cluster,count,number_dimension,paths)

# # 9 Combine all together
encode_train_test_pattern(start_encode_train_data_set, end_encode_train_data_set,start_cluster,max_number_of_cluster,number_dimension,paths)

# # 10 Train and Test with SVRRC
train_test(max_number_of_cluster,paths,number_dimension,n_real,start_gen_real_train_data_set,end_gen_real_train_data_set,fold_num,C_x)

# 11. if you want to know the accuracy of train 
train_test_train(max_number_of_cluster,paths,number_dimension,n_real,start_gen_real_train_data_set,end_gen_real_train_data_set,fold_num,C_x)


      Unnamed: 0  sequence_id      cluster_id
0            559  NC_042413.1  Pezizomycotina
1            505  NC_041489.1  Pezizomycotina
2            434   KY792993.1  Pezizomycotina
3            705  NC_026920.1  Pezizomycotina
4            576   MK318967.1  Pezizomycotina
...          ...          ...             ...
1000        1481   MZ042935.1   Basidiomycota
1001        1553   MN427435.1   Basidiomycota
1002        1466  NC_026782.1   Basidiomycota
1003        1474   LC385608.1   Basidiomycota
1004        1743   KY911090.1   Basidiomycota

[1005 rows x 3 columns]
The cluster distribution is:
{'Pezizomycotina': 335, 'Saccharomycotina': 335, 'Basidiomycota': 335}
No. Sequences: 	 1,005
Min. Length: 	 21,684
Max. Length: 	 99,976
Avg. Length: 	 60,656.88
Min cluster size: 335 (Pezizomycotina, Saccharomycotina, Basidiomycota)
Max cluster size: 335 (Pezizomycotina, Saccharomycotina, Basidiomycota)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,